In [1]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import cPickle, h5py, os, glob

from oc_ica import analysis
reload(analysis)

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 20.0% of memory, cuDNN 4007)


<module 'oc_ica.analysis' from '/home/jesse/Development/oc_ica/oc_ica/analysis.pyc'>

# HDF5 Storage

In [5]:
base_folder = '/home/jesse/Development/results/oc_ica'
n_iter = 10

## mixtures-128_sources-256-k_2

In [6]:
fit_folder = 'mixtures-128_sources-256-k_2'
a_file = 'a_array-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE.h5'
fit_files = ['comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-2.h5',
             'comparison_mixtures-128_sources-256_k-2_priors-L2_L4_RANDOM_COHERENCE_models-3.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-4.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COHERENCE.h5'
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COULOMB.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COULOMB_F.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-RANDOM.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-RANDOM_F.h5',]
sc_fits = ['comparison_first-8-lambdas_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-SC.h5',
           'comparison_last-9-lambdas_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-SC.h5']

models = [f.split('.')[-2].split('-')[-1] for f in fit_files] + [sc_fits[0].split('.')[-2].split('-')[-1]]
print models

['2', '3', '4', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F', 'SC']


In [12]:
n_mixtures = 32
OC = 2.
k = 2
n_sources = int(n_mixtures * OC)
print n_mixtures, n_sources, OC, k
fit_folder = 'mixtures-{}_sources-{}_k-{}'.format(n_mixtures, n_sources, k)
a_file = 'a_array-{}_OC-{}_priors-L2_L4_RANDOM_COHERENCE.h5'.format(n_mixtures, OC)
fit_files = sorted(glob.glob(os.path.join(base_folder, fit_folder, '*.h5')))
sc_fits = None
models = [f.split('.')[-2].split('-')[-1] for f in fit_files]
print models

32 64 2.0 2
['2', '3', '4', 'COHERENCE', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F', 'SC']


## Continue

In [13]:
with h5py.File(os.path.join(base_folder, a_file), 'r') as f:
    A_array = f['A_array'].value
    A_priors = f['A_priors'].value

In [14]:
colors = ['black', 'black', 'black', 'blue', 'red', 'red', 'green', 'green', 'purple']
fmts = ['-', ':', '-.', '-', '-', '--', '-', '--', '-']
with h5py.File(os.path.join(base_folder, fit_folder, fit_files[0]), 'r') as f:
    lambdas = f['lambdas'].value
n_mixtures, n_sources = A_array.shape[2:]
n_iter = A_array.shape[1]

In [15]:
W_fits = np.full((len(A_priors), len(models), lambdas.size, n_iter, n_sources, n_mixtures),
                 np.nan, dtype='float32')
results = np.full((len(A_priors), len(models), lambdas.size, n_iter, 2),
                  np.nan, dtype='float32')
null_results = np.full((len(A_priors), len(models), lambdas.size, (n_iter**2-n_iter)//2, 2),
                  np.nan, dtype='float32')

In [16]:
for ii, f_name in enumerate(fit_files):
    with h5py.File(os.path.join(base_folder, fit_folder, f_name), 'r') as f:
        W_fits[:, ii] = np.squeeze(f['W_fits'])[:, :, :n_iter]

if sc_fits is not None:
    loc = 0
    for ii, f_name in enumerate(sc_fits):
        with h5py.File(os.path.join(base_folder, fit_folder, f_name), 'r') as f:
            n_lambdas = f['W_fits'].shape[2]
            W_fits[:, -1, loc:loc+n_lambdas, :10] = np.squeeze(f['W_fits'])[:, :, :n_iter]
            loc += n_lambdas

In [17]:
reload(analysis)
for ii, p in enumerate(A_priors):
    for jj, m in enumerate(models):
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                try:
                    A = A_array[ii, ll]
                    W = W_fits[ii, jj, kk, ll]
                    assert (not np.isnan(A.sum())) and (not np.isnan(W.sum()))
                    results[ii, jj, kk, ll] = analysis.recovery_statistics_AW(A, W)
                    #results[ii, jj, kk, ll] = [hd, pd, mod]
                    #max_angles[ii, jj, kk, ll] = ma
                    #other_angles[ii, jj, kk, ll] = oa
                except (ValueError, AssertionError):
                    pass

In [18]:
for ii, p in enumerate(A_priors):
    for jj, m in enumerate(models):
        for kk, l in enumerate(lambdas):
            loc = 0
            for ll in range(n_iter):
                for mm in range(ll+1, n_iter):
                    try:
                        A = A_array[ii, ll]
                        W = W_fits[ii, jj, kk, mm]
                        assert (not np.isnan(A.sum())) and (not np.isnan(W.sum()))
                        null_results[ii, jj, kk, loc] = analysis.recovery_statistics_AW(A, W)
                        loc += 1
                    except (ValueError, AssertionError):
                        pass

In [20]:
n_metrics = 2
f, axes = plt.subplots(len(A_priors), n_metrics, figsize=(4*n_metrics, 3*len(A_priors)))
f.suptitle('n_mixtures: {} \nn_sources: {} k: {}'.format(n_mixtures, n_sources, k),
           y=1., fontsize=12)
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    (ae, am) = a
    for jj, m in enumerate(models):
        if ii == 0:
            label = str(m)
            if label != 'SC':
                label = label.title()
            label = label.replace('_Redo', '')
        else:
            label = None

        delta = delta = np.nanmean(results[ii, jj, :, :, 0], axis=1)
        mma = np.nanmean(results[ii, jj, :, :, 1], axis=1)
        ae.semilogx(lambdas, delta, fmts[jj], label=label, c=colors[jj])
        ae.semilogx(lambdas, delta, '.', c=colors[jj])
        ae.minorticks_off()
        am.semilogx(lambdas, mma, fmts[jj], label=label, c=colors[jj])
        am.semilogx(lambdas, mma, '.', c=colors[jj])
        am.minorticks_off()
    ae.legend(loc='upper right', ncol=2, prop={'size': 8})

    ae.set_title(p)
    ae.set_ylabel(r'$\Delta P$')
    ae.set_ylim([0, np.nanmax(results[...,0])])
    am.set_title(p)
    am.set_ylabel(r'Median of $p_{\mathrm{min}}$')
    am.set_ylim([0, 80])
    
ae.set_xlabel(r'$\lambda$')
am.set_xlabel(r'$\lambda$')


f.tight_layout()
plt.savefig('model_fit_comparison-n_mixtures{}-n_sources{}-k{}.pdf'.format(n_mixtures, n_sources, k))
plt.show()
plt.close()

In [147]:
n_metrics = 2
f, axes = plt.subplots(len(A_priors), n_metrics, figsize=(4*n_metrics, 3*len(A_priors)))
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    (ae, am) = a
    for jj, m in enumerate(models):
        if ii == 0:
            label = str(m)
            if label != 'SC':
                label = label.title()
            label = label.replace('_Redo', '')
        else:
            label = None
        delta = np.nanmean(null_results[ii, jj, :, :, 0], axis=1)
        mma = np.nanmean(null_results[ii, jj, :, :, 1], axis=1)

        ae.semilogx(lambdas, delta, fmts[jj], label=label, c=colors[jj])
        ae.semilogx(lambdas, delta, '.', c=colors[jj])
        ae.minorticks_off()

        am.semilogx(lambdas, mma, fmts[jj], label=label, c=colors[jj])
        am.semilogx(lambdas, mma, '.', c=colors[jj])
        am.minorticks_off()

    ae.legend(loc='upper right', ncol=2, prop={'size': 8})

    ae.set_title(p)
    ae.set_ylabel(r'$\Delta P$')
    ae.set_ylim([0, np.nanmax(results[...,0])])
    am.set_title(p)
    am.set_ylabel(r'Median of $p_{\mathrm{min}}$')
    am.set_ylim([0, 80])
    
ae.set_xlabel(r'$\lambda$')
am.set_xlabel(r'$\lambda$')


f.tight_layout()
#plt.savefig('null_model_fit_comparison.pdf')
plt.show()
plt.close()

In [39]:
import matplotlib

In [56]:
n_metrics = 2
colors = [plt.get_cmap('viridis')(x) for x in np.linspace(0, 1, 5)] + ['black']
switch = [0, 9, 4, 9, 2, 9, 3, 9, 5]
fmts = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
f, axes = plt.subplots(1, n_metrics, figsize=(4*2, 3*1))
axes = [axes]
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    (ae, am) = a
    for jj, m in enumerate(models):
        if jj in [0, 2, 4, 6, 8]:
            if ii == 0:
                label = str(m)
                if label != 'SC':
                    label = label.title()
                label = label.replace('_Redo', '')
                if label == '2':
                    label = r'$L_2$'
                if label == '4':
                    label = r'$L_4$'
                if label == 'Random':
                    label = 'Rand. Prior'
            else:
                label = None

            delta = delta = np.nanmean(results[ii, jj, :, :, 0], axis=1)
            mma = np.nanmean(results[ii, jj, :, :, 1], axis=1)
            ae.semilogx(lambdas, delta, fmts[jj], label=label, c=colors[switch[jj]], lw=1.5)
            ae.semilogx(lambdas, delta, '.', c=colors[switch[jj]])
            ae.minorticks_off()
            am.semilogx(lambdas, mma, fmts[jj], label=label, c=colors[switch[jj]], lw=1.5)
            am.semilogx(lambdas, mma, '.', c=colors[switch[jj]])
            am.minorticks_off()
        ae.legend(loc='upper right', ncol=2, prop={'size': 8}, frameon=False)

        ae.set_ylabel(r'$\Delta P$')
        ae.set_ylim([0, np.nanmax(results[...,0])])
        am.set_ylabel(r'Median of $p_{\mathrm{min}}$')
        am.set_ylim([0, 80])
    
ae.set_xlabel(r'$\lambda$')
am.set_xlabel(r'$\lambda$')


f.tight_layout()
plt.savefig('model_fit_comparison-n_mixtures{}-n_sources{}-k{}.pdf'.format(n_mixtures, n_sources, k))
plt.show()
plt.close()